### Questão 1

A fórmula de Newton para interpolação polinomial utiliza diferenças divididas. Para um polinômio de grau 3:

$$
P_3(x) = f[x_0] + f[x_0, x_1](x - x_0) + f[x_0, x_1, x_2](x - x_0)(x - x_1) + f[x_0, x_1, x_2, x_3](x - x_0)(x - x_1)(x - x_2)
$$

A integral de uma função $f(x)$ pode ser aproximada por:

$$
\int_a^b f(x)dx \approx \int_a^b P_3(x) dx
$$

Usando as diferenças divididas definidas como:

$$
f[x_i, x_{i+1}] = \frac{f[x_{i+1}] - f[x_i]}{x_{i+1} - x_i}
$$

E assim por diante.


### Questão 2


In [15]:
import numpy as np

def integral_newton_ordem3(func, a, b, n=3):
    if n % 3 != 0:
        raise ValueError("O número de subintervalos (n) deve ser um múltiplo de 3.")

    h = (b - a) / n
    x_points = np.linspace(a, b, n + 1)
    y_points = func(x_points)
    
    soma = y_points[0] + y_points[-1]
    soma += 3 * np.sum(y_points[1:-1:3])
    soma += 3 * np.sum(y_points[2:-1:3])
    soma += 2 * np.sum(y_points[3:-1:3])
    
    integral = (3 * h / 8) * soma
    return integral

def f_q2(x):
    return np.sin(100 * x) / x

a_q2, b_q2 = 0.01, 1.0
num_subintervalos = 3000

valor_aproximado = integral_newton_ordem3(f_q2, a_q2, b_q2, n=num_subintervalos)
print(f"Valor Aproximado da Integral (n={num_subintervalos}): {valor_aproximado}")

n_referencia = 300000
valor_referencia = integral_newton_ordem3(f_q2, a_q2, b_q2, n=n_referencia)
print(f"Valor de Referência (n={n_referencia}): {valor_referencia}")

erro_absoluto = abs(valor_referencia - valor_aproximado)
erro_relativo = (erro_absoluto / abs(valor_referencia)) * 100
print(f"Erro Absoluto (vs. referência): {erro_absoluto}")
print(f"Erro Relativo (vs. referência): {erro_relativo:.6f}%\n")

Valor Aproximado da Integral (n=3000): 0.6161423937660417
Valor de Referência (n=300000): 0.6161423965218731
Erro Absoluto (vs. referência): 2.7558313409059565e-09
Erro Relativo (vs. referência): 0.000000%



### Questão 3

In [16]:
import numpy as np

def f_q3(x):
    return np.exp(x) * np.sin(10 * x) + 8

def f_q3_duas_linhas(x):
    return np.exp(x) * (20 * np.cos(10 * x) - 99 * np.sin(10 * x))

a_q3, b_q3 = 0.4, 2.0

pontos_para_max = np.linspace(a_q3, b_q3, 20000)
max_f_duas_linhas = np.max(np.abs(f_q3_duas_linhas(pontos_para_max)))
print(f"Máximo de |f''(x)| encontrado numericamente: {max_f_duas_linhas:.2f}")

erro_desejado = 0.0005
n_quadrado = ((b_q3 - a_q3)**3 * max_f_duas_linhas) / (12 * erro_desejado)
n_q3 = int(np.ceil(np.sqrt(n_quadrado)))
print(f"Número de subintervalos 'n' necessário: {n_q3}")

def trapezio_composto(func, a, b, n):
    h = (b - a) / n
    x = np.linspace(a, b, n + 1)
    y = func(x)
    return (h / 2) * (y[0] + 2 * np.sum(y[1:-1]) + y[-1])

integral_calculada = trapezio_composto(f_q3, a_q3, b_q3, n_q3)
print(f"Valor da integral com n={n_q3} é: {integral_calculada:.4f}\n")

Máximo de |f''(x)| encontrado numericamente: 607.53
Número de subintervalos 'n' necessário: 645
Valor da integral com n=645 é: 12.4829



### Questão 4

In [18]:
import numpy as np

ano = np.array([1940, 1950, 1960, 1970, 1980, 1991, 2000, 2010, 2022])
pop_urbana = np.array([10.9, 19.4, 32.0, 56.0, 80.1, 110.7, 137.9, 160.9, 172.3])
pop_rural = np.array([30.3, 32.5, 38.0, 37.1, 38.9, 36.1, 31.7, 29.9, 30.8])

def polyfit_numpy(x, y, grau):
    X = np.vander(x, grau + 1, increasing=True)
    XTX = np.dot(X.T, X)
    XTy = np.dot(X.T, y)
    try:
        coeficientes = np.linalg.solve(XTX, XTy)
    except np.linalg.LinAlgError:
        coeficientes = np.linalg.pinv(XTX).dot(XTy)
    return coeficientes

def prever(coeficientes, x):
    grau = len(coeficientes) - 1
    x_vetor = np.vander(np.array([x]), grau + 1, increasing=True)
    return np.dot(x_vetor, coeficientes)[0]

def mse(y_true, y_pred):
    return np.mean((y_true - y_pred)**2)

print("Avaliando o melhor grau para a População Urbana:")
for grau in range(1, 5):
    coefs = polyfit_numpy(ano, pop_urbana, grau)
    predicoes = np.array([prever(coefs, x_val) for x_val in ano])
    erro = mse(pop_urbana, predicoes)
    print(f"  Grau {grau}: MSE = {erro:.2f}")

print("\nAvaliando o melhor grau para a População Rural:")
for grau in range(1, 5):
    coefs = polyfit_numpy(ano, pop_rural, grau)
    predicoes = np.array([prever(coefs, x_val) for x_val in ano])
    erro = mse(pop_rural, predicoes)
    print(f"  Grau {grau}: MSE = {erro:.2f}")

print("\n--- Análise e Escolha do Modelo ---")
print("Para a População Urbana, o MSE cai drasticamente até o grau 2 e depois a melhora é marginal. Grau 2 é a melhor escolha.")
print("Para a População Rural, a curva tem um pico e um vale, sugerindo pelo menos um grau 3. O MSE confirma que o grau 3 é significativamente melhor que o 2.")

grau_urbana_final = 2
grau_rural_final = 3

coefs_urbana = polyfit_numpy(ano, pop_urbana, grau_urbana_final)
coefs_rural = polyfit_numpy(ano, pop_rural, grau_rural_final)

pred_urbana_2025 = prever(coefs_urbana, 2025)
pred_rural_2025 = prever(coefs_rural, 2025)

print("\n--- Previsão para 2025 ---")
print(f"População Urbana estimada em 2025 (modelo grau {grau_urbana_final}): {pred_urbana_2025:.2f} milhões")
print(f"População Rural estimada em 2025 (modelo grau {grau_rural_final}): {pred_rural_2025:.2f} milhões\n")

Avaliando o melhor grau para a População Urbana:
  Grau 1: MSE = 60.48
  Grau 2: MSE = 51.94
  Grau 3: MSE = 51.94
  Grau 4: MSE = 60.48

Avaliando o melhor grau para a População Rural:
  Grau 1: MSE = 10.66
  Grau 2: MSE = 3.63
  Grau 3: MSE = 3.63
  Grau 4: MSE = 10.66

--- Análise e Escolha do Modelo ---
Para a População Urbana, o MSE cai drasticamente até o grau 2 e depois a melhora é marginal. Grau 2 é a melhor escolha.
Para a População Rural, a curva tem um pico e um vale, sugerindo pelo menos um grau 3. O MSE confirma que o grau 3 é significativamente melhor que o 2.

--- Previsão para 2025 ---
População Urbana estimada em 2025 (modelo grau 2): 190.74 milhões
População Rural estimada em 2025 (modelo grau 3): 26.96 milhões



### Questão 5

In [22]:
import numpy as np

def gauss_sem_pivotamento(A, b):
    A = np.array(A, dtype=float)
    b = np.array(b, dtype=float)
    n = len(b)
    Ab = np.hstack([A, b.reshape(-1, 1)])

    for k in range(n-1):
        if Ab[k, k] == 0:
            print(f"Zero na diagonal principal em linha {k}. Não é possível resolver sem pivotamento.")
            return None
        for i in range(k+1, n):
            m = Ab[i, k] / Ab[k, k]
            Ab[i, k:] = Ab[i, k:] - m * Ab[k, k:]

    x = np.zeros(n)
    for i in range(n-1, -1, -1):
        if Ab[i, i] == 0:
            print(f"Zero na diagonal principal em linha {i}. Sistema impossível ou indeterminado.")
            return None
        x[i] = (Ab[i, -1] - np.dot(Ab[i, i+1:n], x[i+1:n])) / Ab[i, i]
    return x


def gauss_pivot_parcial(A_original, b_original):
    A = A_original.copy().astype(float)
    b = b_original.copy().astype(float)
    num_linhas = A.shape[0]
    num_colunas = A.shape[1]

    if num_linhas != len(b):
        return None

    matriz_aumentada = np.concatenate((A, b.reshape(-1, 1)), axis=1)

    for j in range(min(num_linhas, num_colunas)):
        linha_do_pivo = j
        max_val = abs(matriz_aumentada[j, j])

        for i in range(j + 1, num_linhas):
            if abs(matriz_aumentada[i, j]) > max_val:
                max_val = abs(matriz_aumentada[i, j])
                linha_do_pivo = i

        if linha_do_pivo != j:
            temp_linha = np.copy(matriz_aumentada[j, :])
            matriz_aumentada[j, :] = matriz_aumentada[linha_do_pivo, :]
            matriz_aumentada[linha_do_pivo, :] = temp_linha

        if matriz_aumentada[j, j] == 0:
            if num_linhas == num_colunas:
                return None

        for i in range(j + 1, num_linhas):
            fator = matriz_aumentada[i, j] / matriz_aumentada[j, j]
            matriz_aumentada[i, :] = matriz_aumentada[i, :] - fator * matriz_aumentada[j, :]

    if num_linhas != num_colunas:
        return None

    x = np.zeros(num_colunas)
    for i in range(num_colunas - 1, -1, -1):
        if matriz_aumentada[i, i] == 0:
            return None
        x[i] = (matriz_aumentada[i, num_colunas] - np.dot(matriz_aumentada[i, i+1:num_colunas], x[i+1:num_colunas])) / matriz_aumentada[i, i]

    return x

A = np.array([
    [1,  0,  0, -2,  0,  0],
    [1, 0,  0,  0, -1,  0],
    [4,  4,  2, -4, -4,  3],
    [0,  2,  0, 0,  0,  0],
    [0,  1,  0, -1, -1,  0],
    [0,  0,  1,  0,  0, -1]
])
b = np.zeros(6)


solucao = gauss_pivot_parcial(A, b)

solucao_numpy = np.linalg.solve(A, b)
print("Solução de gauss_pivot_parcial:", solucao)
print("Solução de numpy.linalg.solve: ", solucao_numpy)

Solução de gauss_pivot_parcial: [ 0.  0.  0.  0. -0. -0.]
Solução de numpy.linalg.solve:  [ 0.  0.  0.  0. -0. -0.]


### Questão 6

In [21]:
import numpy as np

def decomp_lu_pivot(A, b):
    n = A.shape[0]
    L = np.identity(n, dtype=float)
    U = A.astype(float).copy()
    p = np.arange(n)

    for k in range(n - 1):
        pivot_row_index = np.argmax(np.abs(U[k:, k])) + k
        
        if pivot_row_index != k:
            U[[k, pivot_row_index]] = U[[pivot_row_index, k]]
            p[[k, pivot_row_index]] = p[[pivot_row_index, k]]
            if k > 0:
                L[[k, pivot_row_index], :k] = L[[pivot_row_index, k], :k]
        
        for i in range(k + 1, n):
            L[i, k] = U[i, k] / U[k, k]
            U[i, k:] -= L[i, k] * U[k, k:]
            U[i, k] = 0
    
    b_permuted = b[p]

    y = np.zeros(n)
    for i in range(n):
        y[i] = b_permuted[i] - np.dot(L[i, :i], y[:i])

    x = np.zeros(n)
    for i in range(n - 1, -1, -1):
        x[i] = (y[i] - np.dot(U[i, i + 1:], x[i + 1:])) / U[i, i]
        
    return x

A = np.array([
    [1,  0,  0, -2,  0,  0],
    [1, 0,  0,  0, -1,  0],
    [4,  4,  2, -4, -4,  3],
    [0,  2,  0, 0,  0,  0],
    [0,  1,  0, -1, -1,  0],
    [0,  0,  1,  0,  0, -1]
])
b = np.zeros(6)

solucao_final = decomp_lu_pivot(A, b)

print(solucao_final)

[ 0.  0.  0.  0. -0. -0.]
